**Import des librairies**

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


**Test de récupération d'infos sur une page de review**

In [ ]:
url = 'https://www.trustpilot.com/review/www.dugood.org?page=1&stars=5'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

reviews = soup.find_all('div', class_='styles_reviewCardInner__EwDq2')
reviews[0].get_text()

'ICIchoku2 reviewsNG5 days agoI recently re-financed a truck note…I recently re-financed a truck note with DuGood Credit Union. The process was smooth and easy, the loan officer was friendly and accommodating and we received a better rate and saved a year in terms of payments! Thanks for helping us save money.Date of experience: October 27, 2023'

In [ ]:
# On va chercher le nombre de page à parcourir
nb_pages = int(soup.find("a", attrs = {'name' :"pagination-button-last"}).get_text())

In [ ]:
# On va chercher le nom de l'entreprise
entreprise = soup.find("h1").find("span").get_text().strip()

In [ ]:
reviews[0].find(class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text.strip()

'Ichoku'

In [ ]:
reviews[0].find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua").get_text()

'NG'

In [ ]:
int(reviews[0].find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l").get_text().split()[0])

2

In [ ]:
reviews[0].find('time').get('datetime')

'2023-10-27T19:30:24.000Z'

In [ ]:
reviews[0].find(class_="typography_heading-s__f7029 typography_appearance-default__AAY17").get_text()

'I recently re-financed a truck note…'

In [ ]:
reviews[0].find(class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn").get_text()

'I recently re-financed a truck note with DuGood Credit Union. The process was smooth and easy, the loan officer was friendly and accommodating and we received a better rate and saved a year in terms of payments! Thanks for helping us save money.'

**Récupération des infos sur une page entière**

In [ ]:
lieux,pseudos,nbs_review,dates_review,titres_review,textes_review = [],[],[],[],[],[]

for review in reviews :
  lieu = review.find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua").text.strip()
  lieux.append(lieu)

  pseudo = review.find(class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text.strip()
  pseudos.append(pseudo)

  nb_review = int(review.find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l").text.strip().split()[0])
  nbs_review.append(nb_review)

  date_review = review.find('time').get('datetime')
  dates_review.append(date_review)

  titre_review = review.find(class_="typography_heading-s__f7029 typography_appearance-default__AAY17").text.strip()
  titres_review.append(titre_review)

  texte_review = review.find(class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn").text.strip()
  textes_review.append(texte_review)

**Fonction pour récupérer les infos d'une entreprise**

In [13]:
# FONCTION POUR RECUPERER LE NOMBRE DE PAGE EN TRIANT PAR ETOILE

def nb_pages (url, star):
  # Récupère le nombre de page d'une URL filtrée sur un nombre d'étoile
  url_star = url+"?stars="+str(star)
  response_star = requests.get(url_star)
  soup_star = BeautifulSoup(response_star.text, 'html.parser')
  if (soup_star.find("a", attrs = {'name' :"pagination-button-last"})) is None :
    if (soup_star.find("a", attrs = {'name' :"pagination-button-5"})) is None :
      if (soup_star.find("a", attrs = {'name' :"pagination-button-4"})) is None :
        if (soup_star.find("a", attrs = {'name' :"pagination-button-3"})) is None :
          if (soup_star.find("a", attrs = {'name' :"pagination-button-2"})) is None :
            if (soup_star.find("a", attrs = {'name' :"pagination-button-1"})) is None :
              nb_page = 0
            else :
              nb_page = 1
          else :
            nb_page = 2
        else :
          nb_page = 3
      else :
        nb_page = 4
    else :
      nb_page = 5
  else :
    nb_page = int(soup_star.find("a", attrs = {'name' :"pagination-button-last"}).get_text())
  time.sleep(3)
  return nb_page



# FONCTION POUR RECUPERER LES INFOS SUR UNE PAGE EN FONCTION DE L'URL, NOMBRE D'ETOILE ET PAGE

def recup_review_page (url, etoile, page):
  # Récupère les infos d'une URL en fonction du nombre d'étoile, et du numéro de page
  entreprises,notes,lieux,pseudos,nbs_review,dates_review,titres_review,textes_review = [],[],[],[],[],[],[],[]
  if page == 1 :
    url_page = url+'?stars='+str(etoile)
  else :
    url_page = url+'?page='+str(page)+"&stars="+str(etoile)
  #print(url_page)
  response = requests.get(url_page)

  soup_response = BeautifulSoup(response.text, 'html.parser')

  reviews_response = soup_response.find_all('div', class_='styles_reviewCardInner__EwDq2')


  for review in reviews_response :
    entreprises.append(url)

    notes.append(etoile)

    if review.find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua") is None :
      lieux.append("")
    else :
      lieu = review.find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua").text.strip()
      lieux.append(lieu)

    if review.find(class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17") is None :
      pseudos.append("")
    else :
      pseudo = review.find(class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text.strip()
      pseudos.append(pseudo)

    if review.find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l") is None :
      nbs_review.append("")
    else :
      nb_review = int(review.find(class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l").text.strip().split()[0])
      nbs_review.append(nb_review)

    if review.find('time').get('datetime') is None :
      dates_review.append("")
    else :
      date_review = review.find('time').get('datetime')
      dates_review.append(date_review)

    if review.find(class_="typography_heading-s__f7029 typography_appearance-default__AAY17") is None :
      titres_review.append("")
    else :
      titre_review = review.find(class_="typography_heading-s__f7029 typography_appearance-default__AAY17").get_text()
      titres_review.append(titre_review)

    if review.find(class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn") is None :
      textes_review.append("")
    else :
      texte_review = review.find(class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn").text.strip()
      textes_review.append(texte_review)

  df_page = pd.DataFrame(list(zip(entreprises,notes,lieux,pseudos,nbs_review,dates_review,titres_review,textes_review)), columns=["Entreprise","Note","Lieu", "Pseudo", "Nb_Review","Date","Titre","Review"])
  time.sleep(3)
  return df_page


# FONCTION POUR RECUPERER TOUTES LES INFOS D'UNE URL
# UTILISE LES 2 FONCTIONS PRECEDENTES

def recup_review (url):
  df_review = pd.DataFrame(columns=["Entreprise","Note","Lieu", "Pseudo", "Nb_Review","Date","Titre","Review"])
  for etoile in range(1,6):
    nombre_pages = nb_pages(url, etoile)
    print(etoile,"étoiles, pages :",nombre_pages)
    if nombre_pages > 0 :
      for page in range(1,min(nombre_pages+1,6)) :
        df_page = recup_review_page (url, etoile, page)
        df_review_concat = pd.concat([df_review, df_page])
        df_review = df_review_concat
  df_review.reset_index(drop=True, inplace=True)
  return df_review

**Test sur l'entreprise DuGood Credit Union**

In [ ]:
url = 'https://www.trustpilot.com/review/www.dugood.org'

In [ ]:
df_page_review = recup_review(url)
df_page_review

,Note,Lieu,Pseudo,Nb_Review,Date,Titre,Review
0,1,US,Gerald Spence,2,2023-06-30T23:01:04.000Z,I own 1 arce of land on Toledo Bend…,I own 1 arce of land on Toledo Bend lake with ...
1,1,US,Gerald Spence,2,2023-06-23T20:58:34.000Z,Weaponized against ordinary hard…,Weaponized against ordinary hard working middl...
2,1,US,LE Roi De La JUNGLE,1,2023-06-26T12:21:21.000Z,Credit card,Credit card please I'm Adleradlervoltaire1gmai...
3,1,US,Linda,1,2023-05-04T23:44:08.000Z,If I could leave 0 stars,"If I could leave 0 stars, I would. The loan d..."
4,1,US,Tina,3,2021-03-08T22:36:36.000Z,You've got to be kidding me!!!,You've got to be kidding me!!!! $10.00 to get ...
...,...,...,...,...,...,...,...
2216,5,US,George,3,2019-12-06T20:25:29.000Z,Very easy,I was pleased with the easy way it was to get ...
2217,5,US,Lene,6,2019-12-06T19:56:30.000Z,Everything went very smooth,
2218,5,US,Dolores,1,2019-12-01T23:19:33.000Z,Tara with DuGood helped me with…,Tara with DuGood helped me with everything. Al...
2219,5,US,Martha,1,2019-12-01T19:50:05.000Z,I was treated very well and everything…,I was treated very well and everything needed ...


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Note').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('index', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Note')):
  _plot_series(series, series_name, i)
  fig.legend(title='Note', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Date']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Date'}, axis=1)
              .sort_values('Date', ascending=True))
  xs = counted['Date']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('Date', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Note')):
  _plot_series(series, series_name, i)
  fig.legend(title='Note', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Date')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_4['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_5['Note'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_5, x='index', y='Note', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

**Test de récupération d'infos sur une page de listing d'entreprise**

In [5]:
url_bank = 'https://www.trustpilot.com/categories/bank'

In [ ]:
response_bank = requests.get(url_bank)

soup_bank = BeautifulSoup(response_bank.text, 'html.parser')

urls_entreprises = soup_bank.find_all('a',attrs = {'name':"business-unit-card"})
urls_entreprises[0].get('href')

'/review/www.dugood.org'

**Récupération des infos sur une page entière**

In [ ]:
societes = []
for url_entreprise in urls_entreprises :
  societe = url_entreprise.get('href')
  societes.append(societe)

societes

['/review/www.dugood.org',
 '/review/panaceafinancial.com',
 '/review/southendcapital.com',
 '/review/networkcapital.net',
 '/review/www.bankfive.com',
 '/review/www.bankmobile.com',
 '/review/eecu.org',
 '/review/www.leafnow.com',
 '/review/majority.com',
 '/review/greatbasin.org',
 '/review/www.carouselchecks.com',
 '/review/www.lendingtree.com',
 '/review/www.cunj.com',
 '/review/www.liveoakbank.com',
 '/review/www.extravaluechecks.com',
 '/review/novo.co',
 '/review/www.bank5connect.com',
 '/review/servbank.com',
 '/review/baselane.com',
 '/review/merrickbank.com']

In [ ]:
nb_pages(url_bank, 0)

8

**Fonction pour récupérer les urls d'une catégorie**

In [2]:
def recup_urls (url_categorie):
  societes = []
  nombre_page_url = nb_pages(url_categorie, 0)
  for i in range (1,nombre_page_url+1) :
    if i == 1 :
      url_page = url_categorie
    else :
      url_page = url_categorie +'?page='+str(i)
    print(url_page)
    response_bank = requests.get(url_page)
    soup_bank = BeautifulSoup(response_bank.text, 'html.parser')
    urls_entreprises = soup_bank.find_all('a',attrs = {'name':"business-unit-card"})
    for url in urls_entreprises :
      societe = url.get('href')
      societes.append(societe)
    time.sleep(3)
  return societes

**Exemple récupération des urls des sociétés de la catégorie bank**

In [8]:
df_test = recup_urls (url_bank)
df_test

https://www.trustpilot.com/categories/bank
https://www.trustpilot.com/categories/bank?page=2
https://www.trustpilot.com/categories/bank?page=3
https://www.trustpilot.com/categories/bank?page=4
https://www.trustpilot.com/categories/bank?page=5
https://www.trustpilot.com/categories/bank?page=6
https://www.trustpilot.com/categories/bank?page=7
https://www.trustpilot.com/categories/bank?page=8


['/review/www.dugood.org',
 '/review/panaceafinancial.com',
 '/review/southendcapital.com',
 '/review/networkcapital.net',
 '/review/www.bankfive.com',
 '/review/www.bankmobile.com',
 '/review/eecu.org',
 '/review/www.leafnow.com',
 '/review/majority.com',
 '/review/greatbasin.org',
 '/review/www.carouselchecks.com',
 '/review/www.lendingtree.com',
 '/review/www.cunj.com',
 '/review/www.liveoakbank.com',
 '/review/www.extravaluechecks.com',
 '/review/novo.co',
 '/review/www.bank5connect.com',
 '/review/servbank.com',
 '/review/baselane.com',
 '/review/merrickbank.com',
 '/review/westernunion.com',
 '/review/openskycc.com',
 '/review/quontic.com',
 '/review/mercury.com',
 '/review/www.grasshopper.bank',
 '/review/www.laurelroad.com',
 '/review/www.chimebank.com',
 '/review/cit.com',
 '/review/texasbaycu.org',
 '/review/trustpilotdemo.com',
 '/review/zilbank.com',
 '/review/sobanco.com',
 '/review/arapahoecu.org',
 '/review/sofi.com',
 '/review/gatewayfirst.com',
 '/review/oneazcu.com',


**Fonction pour récupérer les reviews d'une catégorie entière**

In [9]:
def recup_review_cat (url_categorie) :
  urls_cat = recup_urls(url_categorie)
  print(urls_cat)
  recup_cat = pd.DataFrame(columns=["Entreprise","Note","Lieu", "Pseudo", "Nb_Review","Date","Titre","Review"])
  for url_entreprise in urls_cat :
    url_total_entreprise = "https://www.trustpilot.com"+url_entreprise
    print(url_total_entreprise)
    recup_cat_concat = pd.concat([recup_cat,recup_review(url_total_entreprise)])
    recup_cat = recup_cat_concat
  recup_cat.reset_index(drop=True, inplace=True)
  return recup_cat

In [14]:
df_bank = recup_review_cat(url_bank)


https://www.trustpilot.com/categories/bank
https://www.trustpilot.com/categories/bank?page=2
https://www.trustpilot.com/categories/bank?page=3
https://www.trustpilot.com/categories/bank?page=4
https://www.trustpilot.com/categories/bank?page=5
https://www.trustpilot.com/categories/bank?page=6
https://www.trustpilot.com/categories/bank?page=7
https://www.trustpilot.com/categories/bank?page=8
['/review/www.dugood.org', '/review/panaceafinancial.com', '/review/southendcapital.com', '/review/networkcapital.net', '/review/www.bankfive.com', '/review/www.bankmobile.com', '/review/eecu.org', '/review/www.leafnow.com', '/review/majority.com', '/review/greatbasin.org', '/review/www.carouselchecks.com', '/review/www.lendingtree.com', '/review/www.cunj.com', '/review/www.liveoakbank.com', '/review/www.extravaluechecks.com', '/review/novo.co', '/review/www.bank5connect.com', '/review/servbank.com', '/review/baselane.com', '/review/merrickbank.com', '/review/westernunion.com', '/review/openskycc.com

In [15]:
df_bank

,Entreprise,Note,Lieu,Pseudo,Nb_Review,Date,Titre,Review
0,https://www.trustpilot.com/review/www.dugood.org,1,US,Gerald Spence,2,2023-06-30T23:01:04.000Z,I own 1 arce of land on Toledo Bend…,I own 1 arce of land on Toledo Bend lake with ...
1,https://www.trustpilot.com/review/www.dugood.org,1,US,Gerald Spence,2,2023-06-23T20:58:34.000Z,Weaponized against ordinary hard…,Weaponized against ordinary hard working middl...
2,https://www.trustpilot.com/review/www.dugood.org,1,US,LE Roi De La JUNGLE,1,2023-06-26T12:21:21.000Z,Credit card,Credit card please I'm Adleradlervoltaire1gmai...
3,https://www.trustpilot.com/review/www.dugood.org,1,US,Linda,1,2023-05-04T23:44:08.000Z,If I could leave 0 stars,"If I could leave 0 stars, I would. The loan d..."
4,https://www.trustpilot.com/review/www.dugood.org,1,US,Tina,3,2021-03-08T22:36:36.000Z,You've got to be kidding me!!!,You've got to be kidding me!!!! $10.00 to get ...
...,...,...,...,...,...,...,...,...
25232,https://www.trustpilot.com/review/pnc.com,5,US,Pat,1,2019-09-25T14:50:30.000Z,Preferred bank for over 25+ year,PNC has been my banking partner for over 25+ y...
25233,https://www.trustpilot.com/review/pnc.com,5,US,Asiyeh Golabchi,6,2020-06-15T01:07:54.000Z,Great customer service,Great customer service! I would like to give 5...
25234,https://www.trustpilot.com/review/pnc.com,5,US,Corey,10,2019-10-15T02:46:05.000Z,Best banking service I ever had!,
25235,https://www.trustpilot.com/review/pnc.com,5,US,Lakeland Resort,1,2021-07-04T03:20:27.000Z,Never had a problem.,
